pandas和内置的Python标准库提供了一组高级的、灵活的、快速的工具，可以让你轻松地将数据规整为想要的格式。

在本章中，我会讨论处理缺失数据、重复数据、字符串操作和其它分析数据转换的工具。下一章，我会关注于用多种方法合并、重塑数据集。

# 7.1 处理缺失数据 Handling Missing Data--------------------------------------------------

pandas的目标之一就是尽量轻松地处理缺失数据。例如，pandas对象的所有描述性统计默认都不包括缺失数据。缺失数据在pandas中呈现的方式有些不完美，但对于大多数用户可以保证功能正常。

对于数值数据，pandas使用浮点值NaN（Not a Number）表示缺失数据**np.nan**

Python内置的**None**值在对象数组中也可以作为NA：

> 表7-1列出了一些关于缺失数据处理的函数。

> dropna | fillna | isnull | notnull |

# 滤除缺失数据

过滤掉缺失数据的办法有很多种。你可以通过pandas.isnull或布尔索引的手工方法，

但**dropna**可能会更实用一些。对于一个Series，dropna返回一个仅含非空数据和索引值的Series：

In [1]:
import pandas as pd

from numpy import nan as NA
data = pd.Series([1,NA,3.5,NA,7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [2]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

而对于DataFrame对象，事情就有点复杂了。你可能希望丢弃全NA或含有NA的行或列。dropna默认丢弃任何含有缺失值的行：

传入**how='all'**将只丢弃全为NA的那些行：

用这种方式丢弃列，还要再传入**axis=1**即可：

另一个滤除DataFrame行的问题涉及时间序列数据。假设你只想留下一部分观测数据，可以用**thresh参数**实现此目的：

In [3]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
   ....:                      [NA, NA, NA], [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [4]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [5]:
data[4] = NA
data
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [6]:
import numpy as np
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,0.754025,NaN,NaN
1,0.625062,NaN,NaN
2,0.323021,NaN,0.736308
3,-0.544718,NaN,-1.115130
4,1.192540,0.379044,-0.289424
5,0.111957,-0.968309,-1.308822
6,-0.612185,0.197840,0.539045


In [7]:
df.dropna()

,0,1,2
4,1.192540,0.379044,-0.289424
5,0.111957,-0.968309,-1.308822
6,-0.612185,0.197840,0.539045


In [8]:
df.dropna(thresh=2)

,0,1,2
2,0.323021,NaN,0.736308
3,-0.544718,NaN,-1.115130
4,1.192540,0.379044,-0.289424
5,0.111957,-0.968309,-1.308822
6,-0.612185,0.197840,0.539045


# 填充缺失数据

你可能不想滤除缺失数据（有可能会丢弃跟它有关的其他数据），而是希望通过其他方式填补那些“空洞”。对于大多数情况而言，**fillna方法**是最主要的函数。

通过一个常数调用fillna就会将缺失值替换为那个常数值：

若是通过一个字典调用fillna，就可以实现对不同的列填充不同的值：

fillna默认会返回新对象，但也可以对现有对象进行就地修改：设置参数**inplace=True**

In [9]:
df.fillna(0)

,0,1,2
0,0.754025,0.000000,0.000000
1,0.625062,0.000000,0.000000
2,0.323021,0.000000,0.736308
3,-0.544718,0.000000,-1.115130
4,1.192540,0.379044,-0.289424
5,0.111957,-0.968309,-1.308822
6,-0.612185,0.197840,0.539045


In [10]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,0.754025,0.500000,0.000000
1,0.625062,0.500000,0.000000
2,0.323021,0.500000,0.736308
3,-0.544718,0.500000,-1.115130
4,1.192540,0.379044,-0.289424
5,0.111957,-0.968309,-1.308822
6,-0.612185,0.197840,0.539045


In [11]:
df.fillna(0, inplace=True)
df

,0,1,2
0,0.754025,0.000000,0.000000
1,0.625062,0.000000,0.000000
2,0.323021,0.000000,0.736308
3,-0.544718,0.000000,-1.115130
4,1.192540,0.379044,-0.289424
5,0.111957,-0.968309,-1.308822
6,-0.612185,0.197840,0.539045


对reindexing有效的那些插值方法也可用于fillna：**参数method**，**参数limit**

In [12]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,-1.379232,-0.704321,-1.775693
1,-0.490733,1.383172,0.186868
2,0.379621,NaN,-0.727550
3,-0.825950,NaN,-0.506310
4,0.058374,NaN,NaN
5,1.970307,NaN,NaN


In [13]:
df.fillna(method='ffill')

,0,1,2
0,-1.379232,-0.704321,-1.775693
1,-0.490733,1.383172,0.186868
2,0.379621,1.383172,-0.727550
3,-0.825950,1.383172,-0.506310
4,0.058374,1.383172,-0.506310
5,1.970307,1.383172,-0.506310


In [14]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-1.379232,-0.704321,-1.775693
1,-0.490733,1.383172,0.186868
2,0.379621,1.383172,-0.727550
3,-0.825950,1.383172,-0.506310
4,0.058374,NaN,-0.506310
5,1.970307,NaN,-0.506310


只要有些创新，你就可以利用fillna实现许多别的功能。比如说，你可以传入Series的平均值或中位数：

In [15]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

> 表7-2列出了fillna的参考。

> value | method | axis | inplace | limit |

# 7.2 数据转换 Data Transformation-----------------------------------------------------------

本章到目前为止介绍的都是数据的重排。另一类重要操作则是过滤、清理以及其他的转换工作。

# 移除重复数据

DataFrame中出现重复行有多种原因。下面就是一个例子：

DataFrame的**duplicated方法**返回一个布尔型Series，表示各行是否是重复行（前面出现过的行）：【默认是全部列要完全相同】

还有一个与此相关的**drop_duplicates方法**，它会返回一个DataFrame，重复的数组去除

In [16]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
   ....:                      'k2': [1, 1, 2, 3, 3, 4, 4]})
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [17]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


这两个方法默认会判断全部列，你也可以指定部分列进行重复项判断。假设我们还有一列值，且只希望根据k1列过滤重复项：**指定列**

duplicated和drop_duplicates默认保留的是第一个出现的值组合。传入**keep='last'**则保留最后一个：

In [18]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [19]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [20]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


# 利用函数或映射进行数据转换

对于许多数据集，你可能希望根据数组、Series或DataFrame列中的值来实现转换工作。我们来看看下面这组有关肉类的数据：

假设你想要添加一列表示该肉类食物来源的动物类型。我们先编写一个不同肉类到动物的映射：

Series的map方法可以接受一个函数或含有映射关系的字典型对象，但是这里有一个小问题，即有些肉类的首字母大写了，而另一些则没有。因此，我们还需要使用**Series的str.lower方法**，将各个值转换为小写：

我们也可以传入一个能够完成全部这些工作的匿名函数：

使用**map方法**是一种*实现元素级转换以及其他数据清理工作的便捷方式*。

In [21]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
   ....:                               'Pastrami', 'corned beef', 'Bacon',
   ....:                               'pastrami', 'honey ham', 'nova lox'],
   ....:                      'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [22]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [23]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [24]:
data['animal'] = data['food'].map(lambda x: meat_to_animal[x.lower()])    # 此处x是迭代出来的元素
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


# 替换值

利用fillna方法填充缺失数据可以看做值替换的一种特殊情况。前面已经看到，map可用于修改对象的数据子集，

而**replace方法**则提供了一种实现该功能的更简单、更灵活的方式。我们来看看下面这个Series：

-999这个值可能是一个表示缺失数据的标记值。要将其替换为pandas能够理解的NA值，我们可以利用replace来产生一个新的Series（除非传入inplace=True）：

如果你希望一次性替换多个值，可以传入一个由待替换值组成的列表以及一个替换值：

要让每个值有不同的替换值，可以**传递一个替换列表**：传入的参数也可以是字典：

In [25]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

In [26]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [27]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [28]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

笔记：data.replace方法与data.str.replace不同，后者做的是字符串的元素级替换。我们会在后面学习Series的字符串方法。

# 重命名轴索引

跟Series中的值一样，轴标签也可以通过函数或映射进行转换，从而得到一个新的不同标签的对象。轴还可以被就地修改，而无需新建一个数据结构。接下来看看下面这个简单的例子：

跟Series一样，轴索引也有一个**map方法**：

In [29]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
   ....:                     index=['Ohio', 'Colorado', 'New York'],
   ....:                     columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [30]:
transform = lambda x : x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

你可以将其赋值给index，这样就可以对DataFrame进行就地修改：

如果想要创建**数据集的转换版**（而不是修改原始数据），比较实用的方法是DataFrame对象的**rename方法**：【赋值index参数，columns参数】

特别说明一下，rename可以结合字典型对象实现对指定轴标签的更新：

rename可以实现复制DataFrame并对其索引和列标签进行赋值。如果希望就地修改某个数据集，传入inplace=True即可：

In [31]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [32]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [33]:
data.rename(index={'ÓHIO':'INDIANA'}, columns={'three':'peekaboo'})

,one,two,peekaboo,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


# 离散化和面元划分Discretization and Binning

为了便于分析，连续数据常常被离散化或拆分为“面元”（bin）。假设有一组人员数据，而你希望将它们划分为不同的年龄组：

接下来将这些数据划分为“18到25”、“26到35”、“35到60”以及“60以上”几个面元。要实现该功能，你需要使用pandas的**cut函数**：

In [34]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18,25,35,60,100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

pandas返回的是一个特殊的**Categorical对象**。结果展示了pandas.cut划分的面元。你可以将其看做一组表示面元名称的字符串。

它的底层含有一个表示不同分类名称的类型数组，以及一个codes属性中的年龄数据的标签：

pd.value_counts(cats)是pandas.cut结果的面元计数。

In [35]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [36]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [37]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

跟“区间”的数学符号一样，圆括号表示开端，而方括号则表示闭端（包括）。哪边是闭端可以通过cut函数的**参数right=False**进行修改：

你可 以通过传递一个列表或数组到**labels参数**，设置自己的面元名称：

In [38]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [39]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [MiddleAged < Senior < YoungAdult < Youth]

如果向cut传入的是面元的数量而不是确切的面元边界，则它会根据数据的最小值和最大值计算**等长面元**。下面这个例子中，我们将一些均匀分布的数据分成四组：

**参数precision=2**，限定小数只有两位。

In [40]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.73, 0.96], (0.5, 0.73], (0.27, 0.5], (0.044, 0.27], (0.27, 0.5], ..., (0.73, 0.96], (0.27, 0.5], (0.044, 0.27], (0.73, 0.96], (0.044, 0.27]]
Length: 20
Categories (4, interval[float64]): [(0.044, 0.27] < (0.27, 0.5] < (0.5, 0.73] < (0.73, 0.96]]

**qcut函数**是一个非常类似于cut的函数，它可以根据样本分位数对数据进行面元划分。

根据数据的分布情况，cut可能无法使各个面元中含有相同数量的数据点。而qcut由于使用的是样本分位数，因此可以得到大小基本相等的面元：

与cut类似，你也可以传递自定义的分位数（0到1之间的数值，包含端点）：

本章稍后在讲解聚合和分组运算时会再次用到cut和qcut，因为这两个离散化函数对分位和分组分析非常重要。

In [41]:
data = np.random.randn(1000)
cats = pd.qcut(data, 4)
cats

[(0.668, 3.544], (0.668, 3.544], (-3.49, -0.673], (0.668, 3.544], (-0.673, -0.0233], ..., (-0.0233, 0.668], (-0.0233, 0.668], (-0.0233, 0.668], (-3.49, -0.673], (-0.673, -0.0233]]
Length: 1000
Categories (4, interval[float64]): [(-3.49, -0.673] < (-0.673, -0.0233] < (-0.0233, 0.668] < (0.668, 3.544]]

In [42]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(1.349, 3.544], (-0.0233, 1.349], (-1.208, -0.0233], (1.349, 3.544], (-1.208, -0.0233], ..., (-0.0233, 1.349], (-0.0233, 1.349], (-0.0233, 1.349], (-1.208, -0.0233], (-1.208, -0.0233]]
Length: 1000
Categories (4, interval[float64]): [(-3.49, -1.208] < (-1.208, -0.0233] < (-0.0233, 1.349] < (1.349, 3.544]]

# 检测和过滤异常值

过滤或变换异常值outlier，在很大程度上就是运用数组运算。来看一个含有正态分布数据的DataFrame：

假设你想要找出某列中绝对值大小超过3的值：

要选出全部含有“超过3或－3的值”的行，你可以在布尔型DataFrame中使用**any方法**：

根据这些条件，就可以对值进行设置。下面的代码可以将值限制在区间－3到3以内：

根据数据的值是正还是负，np.sign(data)可以生成1和-1：

In [43]:
pd.options.display.max_rows=10
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.014601,-0.072307,-0.006952,0.087490
std,1.032855,0.992999,1.001301,1.016484
min,-3.710175,-2.903373,-3.502410,-2.966477
25%,-0.690795,-0.781222,-0.659963,-0.605314
50%,-0.029686,-0.088452,0.047079,0.087189
75%,0.688793,0.604373,0.666803,0.791058
max,3.447659,2.967978,3.053390,3.245497


In [44]:
col = data[2]
col[np.abs(col)>3]

244   -3.502410
506   -3.217324
567    3.053390
Name: 2, dtype: float64

In [45]:
data[(np.abs(data)>3).any(axis=1)]

,0,1,2,3
50,-3.710175,0.705618,-0.483052,1.706480
83,-0.921601,-1.561984,0.052623,3.245497
243,1.291699,0.259232,1.071449,3.087385
244,0.697713,-0.845417,-3.502410,0.600993
266,3.447659,0.998138,0.108204,1.025115
410,-3.184161,0.680485,0.732042,0.524832
506,0.025376,1.866919,-3.217324,-2.253782
567,0.773618,0.936024,3.053390,1.565887
601,-3.556331,0.145475,-1.006922,-0.447805
671,-3.690144,-1.133619,0.936519,-0.163880


In [46]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.012907,-0.072307,-0.006286,0.087157
std,1.024580,0.992999,0.998832,1.015495
min,-3.000000,-2.903373,-3.000000,-2.966477
25%,-0.690795,-0.781222,-0.659963,-0.605314
50%,-0.029686,-0.088452,0.047079,0.087189
75%,0.688793,0.604373,0.666803,0.791058
max,3.000000,2.967978,3.000000,3.000000


In [47]:
np.sign(data).head()

,0,1,2,3
0,-1.0,1.0,-1.0,-1.0
1,1.0,-1.0,1.0,1.0
2,-1.0,-1.0,-1.0,-1.0
3,-1.0,1.0,-1.0,1.0
4,1.0,-1.0,-1.0,-1.0


# 排列和随机采样Permutation and Random Sampling

利用numpy.random.**permutation函数**可以轻松实现对Series或DataFrame的列的排列工作（**permuting，随机重排序**）。通过需要排列的轴的长度调用permutation，可产生一个表示新顺序的整数数组：

'''函数文档

    Randomly permute a sequence, or return a permuted range.

    If `x` is a multi-dimensional array, it is only shuffled along its
    first index.

    Parameters
    ----------
    x : int or array_like
        If `x` is an integer, randomly permute ``np.arange(x)``.
        If `x` is an array, make a copy and shuffle the elements
        randomly.
'''

然后就可以在基于iloc的索引操作或**take函数**中使用该数组了：

In [48]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [49]:
sampler = np.random.permutation(5)
sampler

array([0, 1, 2, 4, 3])

In [50]:
df.take(sampler)

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
4,16,17,18,19
3,12,13,14,15


如果不想用替换的方式选取随机子集，可以在Series和DataFrame上使用**sample方法**：

要通过替换的方式产生样本（允许重复选择），可以传递参数replace=True：

In [51]:
df.sample(n=3)

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
4,16,17,18,19


# 计算指标/哑变量Computing Indicator/Dummy Variables

另一种常用于统计建模或机器学习的转换方式是：将分类变量（categorical variable）转换为“哑变量”或“指标矩阵”。

如果DataFrame的某一列中含有k个不同的值，则可以派生出一个k列矩阵或DataFrame（其值全为1和0）。

pandas有一个**get_dummies函数**可以实现该功能（其实自己动手做一个也不难）。使用之前的一个DataFrame例子：

In [52]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
   .....:                    'data1': range(6)})
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [53]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [54]:
pd.get_dummies(df['data1'])

,0,1,2,3,4,5
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,1,0,0,0
3,0,0,0,1,0,0
4,0,0,0,0,1,0
5,0,0,0,0,0,1


有时候，你可能想给指标DataFrame的列加上一个前缀，以便能够跟其他数据进行合并。get_dummies的**prefix参数**可以实现该功能：

In [55]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)    # 注意df[['data1']]返回DataFrame类型，而df['data1']返回Series类型【只取到一个列】
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


如果DataFrame中的某行同属于多个分类，则事情就会有点复杂。看一下MovieLens 1M数据集，14章会更深入地研究它：

要为每个genre添加指标变量就需要做一些数据规整操作。首先，我们从数据集中抽取出不同的genre值：

In [56]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::', header=None, names=mnames)
movies

D:\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [57]:
all_genres = []
for x in movies['genres']:
    all_genres.extend(x.split('|'))
genres = np.unique(all_genres)
genres

array(['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], 
      dtype='<U11')

构建indicator DataFrame的方法之一是从一个全零DataFrame开始：

现在，迭代每一部电影，并将dummies各行的条目设为1。要这么做，我们使用dummies.columns来计算每个类型的列索引：

然后，根据索引，使用.iloc设定值：

然后，和以前一样，再将其与movies合并起来：

In [58]:
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
gen = movies.genres[0]
gen.split('|')
dummies.columns.get_indexer(gen.split('|'))

array([2, 3, 4], dtype=int32)

In [60]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1
dummies

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic

,movie_id,title,genres,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Crime,Genre_Documentary,...,Genre_Fantasy,Genre_Film-Noir,Genre_Horror,Genre_Musical,Genre_Mystery,Genre_Romance,Genre_Sci-Fi,Genre_Thriller,Genre_War,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents (2000),Comedy,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,3949,Requiem for a Dream (2000),Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,3950,Tigerland (2000),Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,3951,Two Family House (2000),Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


笔记：对于很大的数据，用这种方式构建多成员指标变量就会变得非常慢。最好使用更低级的函数，将其写入NumPy数组，然后结果包装在DataFrame中。

一个对统计应用有用的秘诀是：结合get_dummies和诸如cut之类的离散化函数：

In [62]:
np.random.seed(12345)
values = np.random.rand(10)
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values,bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


# 7.3 字符串操作--------------------------------------------------------------------------------------

大部分文本运算都直接做成了字符串对象的内置方法。对于更为复杂的模式匹配和文本操作，则可能需要用到正则表达式。pandas对此进行了加强，它使你能够对整组数据应用字符串表达式和正则表达式，而且能处理烦人的缺失数据。

# 字符串对象方法

对于许多字符串处理和脚本应用，内置的字符串方法已经能够满足要求了。例如，以逗号分隔的字符串可以用split拆分成数段：

split常常与strip一起使用，以去除空白符（包括换行符）：

利用加法，可以将这些子字符串以双冒号分隔符的形式连接起来：

In [63]:
val = 'a,b,  guido'
pieces = [x.strip() for x in val.split(',')]
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

但这种方式并不是很实用。一种更快更符合Python风格的方式是，向字符串"::"的join方法传入一个列表或元组：

In [64]:
'::'.join(pieces)

'a::b::guido'

其它方法关注的是子串定位。检测子串的最佳方式是利用Python的in关键字，还可以使用index和find：

注意find和index的区别：如果找不到字符串，index将会引发一个异常（而不是返回－1）

与此相关，count可以返回指定子串的出现次数：

replace用于将指定模式替换为另一个模式。通过传入空字符串，它也常常用于删除模式：

> 表7-3列出了Python内置的字符串方法。

> count | endswith startswith | join | index  find  rfind |replace | strip rstrip lstrip |split | lower upper | ljust rjust |

# 正则表达式

Python内置的re模块负责对字符串应用正则表达式。我将通过一些例子说明其使用方法。

re模块的函数可以分为三个大类：模式匹配、替换以及拆分。当然，它们之间是相辅相成的。一个regex描述了需要在文本中定位的一个模式，它可以用于许多目的。我们先来看一个简单的例子：假设我想要拆分一个字符串，分隔符为数量不定的一组空白符（制表符、空格、换行符等）。描述一个或多个空白符的regex是\s+：

# pandas的矢量化字符串函数Vectorized String Functions in pandas

通过data.map，所有字符串和正则表达式方法都能被应用于（传入lambda表达式或其他函数）各个值，但是如果存在NA（null）就会报错。

为了解决这个问题，Series有一些能够跳过NA值的面向数组方法，进行字符串操作。通过**Series的str属性**即可访问这些方法。

例如，我们可以通过str.contains检查各个电子邮件地址是否含有"gmail"：

也可以使用正则表达式，还可以加上任意re选项（如IGNORECASE）：

In [65]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
   .....:         'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data.str.contains('gmail')

Dave     False
Rob       True
Steve     True
Wes        NaN
dtype: object

In [66]:
import re
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Rob        [(rob, gmail, com)]
Steve    [(steve, gmail, com)]
Wes                        NaN
dtype: object

有两个办法可以实现矢量化的元素获取操作：要么使用**str.get方法**，要么获取**str属性使用索引**：

In [67]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Rob      True
Steve    True
Wes       NaN
dtype: object

In [68]:
matches.str.get(1)

Dave    NaN
Rob     NaN
Steve   NaN
Wes     NaN
dtype: float64

In [69]:
matches.str[0]

Dave    NaN
Rob     NaN
Steve   NaN
Wes     NaN
dtype: float64

In [70]:
data.str[:5]

Dave     dave@
Rob      rob@g
Steve    steve
Wes        NaN
dtype: object